In [1]:
from figen_class12 import FiGen


In [2]:
!pip install lightgbm

In [3]:
!conda install -c conda-forge lightgbm 

Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.11.1
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0



## Package Plan ##

  environment location: /Users/jangsehwan/opt/anaconda3

  added / updated specs:
    - lightgbm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1w             |       h8a1eda9_0         1.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.7 MB

The following packages will be UPDATED:

  openssl                                 1.1.1v-h8a1eda9_0 --> 1.1.1w-h8a1eda9_0 


Proceed ([y]/n)? ^C

CondaSystemExit: 
O

In [ ]:
!pip install category_encoders

In [ ]:
!pip install imblearn.under_sampling
!pip install imblearn.ensemble

<임밸런스드 방법론>

1. 언더샘플링
2. 오버샘플링

3. 언더샘플링+앙상블
4. ctgan
5. ctgan+앙상블
6. FDDHS
7. FDDHS+앙상블


<인코딩 방법론>

1. 범주형 변수 인코딩후 생성 -> 모델 적합후 성능 비교
2. 범주형 변수 생성후 인코딩  -> 모델 적합후 성능 비교


# 다음주 월요일 저녁 -> 본부장님, 상준님 

In [10]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder, WOEEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedRandomForestClassifier
from sdv.single_table import CTGANSynthesizer




class imblanced_compare:
    
    def __init__(self, data_path, categorical_columns:list):
        self.data_path = data_path
        self.categorical_columns = categorical_columns

    def load_dataset(file_path):
        data = pd.read_csv(file_path)
        X = data.drop(columns=['target'])
        y = data['target']
        return X, y

    def split_data(X, y, test_size=0.2, random_state=1004):
        return train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    def under_sampling(X_train, y_train):
        rus = RandomUnderSampler()
        X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
        return X_resampled, y_resampled
    
    def ensemble_under_sampling(X_train, y_train):
        brf = BalancedRandomForestClassifier()
        brf.fit(X_train, y_train)
        return brf
    
    def generate_synthetic_data(self,X_resampled, y_resampled):
        figen = FiGen(ratio=0.3, categorical_columns=self.categorical_columns)
        synthetic_X, synthetic_y = figen.fit(X_resampled, y_resampled, X_train, y_train)
        return synthetic_X, synthetic_y
    
    def encode_data(X_train, y_train):
        ohe = OneHotEncoder()
        X_ohe = ohe.fit_transform(X_train[categorical_columns])
    
        mean_encoder = TargetEncoder()
        X_mean_encoded = mean_encoder.fit_transform(X_train[categorical_columns], y_train)
    
        woe_encoder = WOEEncoder()
        X_woe_encoded = woe_encoder.fit_transform(X_train[categorical_columns], y_train)
    
        return X_ohe, X_mean_encoded, X_woe_encoded
    
    def evaluate_model(model, X_train, y_train, X_test, y_test):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        roc_auc = roc_auc_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        return roc_auc, f1, classification_report(y_test, y_pred)
    
    def fit(self,data, categorical_columns:list):
        # Load and preprocess data
        X, y = self.load_dataset(data)
        X_train, X_test, y_train, y_test = split_data(X, y)
    
        # Create LGBM model
        lgbm_model = LGBMClassifier()
    
        # Under Sampling
        X_resampled, y_resampled = self.under_sampling(X_train, y_train)
        roc_auc_us, f1_us, report_us = self.evaluate_model(lgbm_model, X_resampled, y_resampled, X_test, y_test)
    
        # Ensemble with Under Sampling
        brf = self.ensemble_under_sampling(X_train, y_train)
        roc_auc_brf, f1_brf, report_brf = self.evaluate_model(brf, X_train, y_train, X_test, y_test)
    
        # Generate Synthetic Data
        synthetic_X, synthetic_y = self.generate_synthetic_data(X_resampled, y_resampled)
        roc_auc_syn, f1_syn, report_syn = self.evaluate_model(lgbm_model, synthetic_X, synthetic_y, X_test, y_test)
    
        # Encode Data
        X_ohe, X_mean_encoded, X_woe_encoded = self.encode_data(X_train, y_train)
        roc_auc_ohe, f1_ohe, report_ohe = self.evaluate_model(lgbm_model, X_ohe, y_train, X_test, y_test)
        roc_auc_mean_encoded, f1_mean_encoded, report_mean_encoded = self.evaluate_model(lgbm_model, X_mean_encoded, y_train, X_test, y_test)
        roc_auc_woe_encoded, f1_woe_encoded, report_woe_encoded = self.evaluate_model(lgbm_model, X_woe_encoded, y_train, X_test, y_test)
    
        # Print Results
        print("Under Sampling Results:")
        print("ROC AUC:", roc_auc_us)
        print("F1 Score:", f1_us)
        print(report_us)
    
        print("\nBalanced Random Forest Results:")
        print("ROC AUC:", roc_auc_brf)
        print("F1 Score:", f1_brf)
        print(report_brf)
    
        print("\nSynthetic Data (FiGen) Results:")
        print("ROC AUC:", roc_auc_syn)
        print("F1 Score:", f1_syn)
        print(report_syn)
    
        print("\nOne-Hot Encoding Results:")
        print("ROC AUC:", roc_auc_ohe)
        print("F1 Score:", f1_ohe)
        print(report_ohe)
    
        print("\nMean Encoding Results:")
        print("ROC AUC:", roc_auc_mean_encoded)
        print("F1 Score:", f1_mean_encoded)
        print(report_mean_encoded)
    
        print("\nWOE Encoding Results:")
        print("ROC AUC:", roc_auc_woe_encoded)
        print("F1 Score:", f1_woe_encoded)
        print(report_woe_encoded)
    


In [ ]:
test = ImbalancedCompare(data_path='your_data.csv', categorical_columns=['cat_col1', 'cat_col2'])
test.fit()

In [20]:
import numpy as np
import pandas as pd
from typing import List, Tuple
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder, WOEEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedRandomForestClassifier
from sdv.single_table import CTGANSynthesizer
from figen_class12 import FiGen  # Assuming FiGen is imported from figen_class

class ImbalancedCompare_lib:
    
    def __init__(self, categorical_columns: List[str], generation_rate: float):
        self.categorical_columns = categorical_columns
        self.generation_rate = generation_rate


    def split_data(self, X: pd.DataFrame, y: pd.Series, test_size: float = None, random_state: int = 1004) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
        """
        Split the dataset into train and test sets.

        Args:
            X (pd.DataFrame): Feature matrix.
            y (pd.Series): Target variable.
            test_size (float): Proportion of the dataset to include in the test split.
            random_state (int): Seed for random number generator.

        Returns:
            X_train (pd.DataFrame): Training features.
            X_test (pd.DataFrame): Testing features.
            y_train (pd.Series): Training target.
            y_test (pd.Series): Testing target.
        """
        if test_size is None:
            test_size = self.generation_rate
        
        return train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    def under_sampling(self, X_train: pd.DataFrame, y_train: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:
        """
        Apply random under-sampling to balance the classes.

        Args:
            X_train (pd.DataFrame): Training features.
            y_train (pd.Series): Training target.

        Returns:
            X_resampled (pd.DataFrame): Resampled features.
            y_resampled (pd.Series): Resampled target.
        """
        rus = RandomUnderSampler()
        X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
        return X_resampled, y_resampled
    
    # ... (Other functions)

    def generate_synthetic_data(self, X_resampled: pd.DataFrame, y_resampled: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:
        """
        Generate synthetic data using FiGen.

        Args:
            X_resampled (pd.DataFrame): Resampled features.
            y_resampled (pd.Series): Resampled target.

        Returns:
            synthetic_X (pd.DataFrame): Synthetic features.
            synthetic_y (pd.Series): Synthetic target.
        """
        figen = FiGen(ratio=0.3, categorical_columns=self.categorical_columns)
        synthetic_X, synthetic_y = figen.fit(X_resampled, y_resampled, X_train, y_train)
        return synthetic_X, synthetic_y
    
        # 모델 평가 함수 (추가)
    def evaluate_model(self, model, X_train, y_train, X_test, y_test):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        roc_auc = roc_auc_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        return roc_auc, f1, report
    
    def fit(self, X,y):
        """
        Fit the model and evaluate various techniques.

        Args:
            data (str): Path to the dataset file.
        """
        # Load and preprocess data
        X_train, X_test, y_train, y_test = self.split_data(X, y)
    
        # Create LGBM model
        lgbm_model = LGBMClassifier()
    
        # Under Sampling
        X_resampled, y_resampled = self.under_sampling(X_train, y_train)
        roc_auc_us, f1_us, report_us = self.evaluate_model(lgbm_model, X_resampled, y_resampled, X_test, y_test)
    
        # Ensemble with Under Sampling
        brf = self.ensemble_under_sampling(X_train, y_train)
        roc_auc_brf, f1_brf, report_brf = self.evaluate_model(brf, X_train, y_train, X_test, y_test)
    
        # Generate Synthetic Data
        synthetic_X, synthetic_y = self.generate_synthetic_data(X_resampled, y_resampled)
        roc_auc_syn, f1_syn, report_syn = self.evaluate_model(lgbm_model, synthetic_X, synthetic_y, X_test, y_test)
    
         # Encode Data
        X_ohe, X_mean_encoded, X_woe_encoded = self.encode_data(X_train, y_train)
        roc_auc_ohe, f1_ohe, report_ohe = self.evaluate_model(lgbm_model, X_ohe, y_train, X_test, y_test)
        roc_auc_mean_encoded, f1_mean_encoded, report_mean_encoded = self.evaluate_model(lgbm_model, X_mean_encoded, y_train, X_test, y_test)
        roc_auc_woe_encoded, f1_woe_encoded, report_woe_encoded = self.evaluate_model(lgbm_model, X_woe_encoded, y_train, X_test, y_test)
    
        # Print Results
        print("Under Sampling Results:")
        print("ROC AUC:", roc_auc_us)
        print("F1 Score:", f1_us)
        print(report_us)
    
        print("\nBalanced Random Forest Results:")
        print("ROC AUC:", roc_auc_brf)
        print("F1 Score:", f1_brf)
        print(report_brf)
    
        print("\nSynthetic Data (FiGen) Results:")
        print("ROC AUC:", roc_auc_syn)
        print("F1 Score:", f1_syn)
        print(report_syn)
    
        print("\nOne-Hot Encoding Results:")
        print("ROC AUC:", roc_auc_ohe)
        print("F1 Score:", f1_ohe)
        print(report_ohe)
    
        print("\nMean Encoding Results:")
        print("ROC AUC:", roc_auc_mean_encoded)
        print("F1 Score:", f1_mean_encoded)
        print(report_mean_encoded)
    
        print("\nWOE Encoding Results:")
        print("ROC AUC:", roc_auc_woe_encoded)
        print("F1 Score:", f1_woe_encoded)
        print(report_woe_encoded)
    




In [9]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
# data (as pandas dataframes) 
X = statlog_german_credit_data.data.features 
y = statlog_german_credit_data.data.targets 

# metadata 
print(statlog_german_credit_data.metadata) 
  
# variable information 
print(statlog_german_credit_data.variables) 


{'uci_id': 144, 'name': 'Statlog (German Credit Data)', 'repository_url': 'https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data', 'data_url': 'https://archive.ics.uci.edu/static/public/144/data.csv', 'abstract': 'This dataset classifies people described by a set of attributes as good or bad credit risks. Comes in two formats (one all numeric). Also comes with a cost matrix', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1000, 'num_features': 20, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Other', 'Marital Status', 'Age', 'Occupation'], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1994, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5NC77', 'creators': ['Hans Hofmann'], 'intro_paper': None, 'additional_info': {'summary': 'Two datasets are provided.  the original dataset, in the form provided by

In [21]:
# 범주형 변수의 컬럼명을 저장할 빈 리스트 생성
categorical_columns_list = []

# 데이터프레임의 컬럼을 순회하면서 데이터 유형을 확인
for column in X.columns:
    if X[column].dtype == 'object':  # 데이터 유형이 'object'인 경우 (범주형 변수)
        categorical_columns_list.append(column) 


# 클래스의 인스턴스를 생성합니다.
comparator = ImbalancedCompare_lib( categorical_columns=categorical_columns_list, generation_rate=0.3)

# 모델을 학습하고 다양한 기술을 평가합니다.
comparator.fit(X, y)


AttributeError: 'ImbalancedCompare_lib' object has no attribute 'evaluate_model'

# DDHS

1. 데이터 다운 (완)
2. 모델 피팅 파이프라인 수정
3. Figen 에러 고치기
4. 1회 돌려보기
5. 해당 내용 커밋
6. 10.25 결과 리뷰